In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
from matplotlib import pyplot as plt
import scipy.cluster.hierarchy as sch
import random
from sklearn.decomposition import PCA

This code serves as an example of filtering, it is obsolete after the data is filtered.  \
The filtered data is shared in this GitHub repository. 

In [ ]:
# Read assay files and remove assays listed as duplicates or private
# The assay files below are NOT shared

assay_train_file = '../data/assay_matrix_discrete_train_scaff_filt.csv'
assay_test_file = '../data/assay_matrix_discrete_test_scaff.csv'
assay_train_df = pd.read_csv(assay_train_file).fillna(-1)
assay_test_df = pd.read_csv(assay_test_file).fillna(-1)
assay_all_df = pd.concat([assay_train_df, assay_test_df])

In [ ]:
assay_all_np = assay_all_df.to_numpy()[:, 1:].astype('int64')
assay_all_np

In [ ]:
assay_all_df.reset_index(drop=True, inplace=True)

In [ ]:
assay_all_df

In [ ]:
# load PAINS indicies
pains_filtered_file = '../data/compound_analysis.npz'
with open(pains_filtered_file, "rb") as data:
    np_obj = np.load(data) 
    pains_overall = np_obj['pains_overall']

In [ ]:
# remove PAINS
mask = np.zeros(16978, dtype=bool)
mask[pains_overall] = True
assays_np_nopains = assay_all_np[~mask]

smiles_np = assay_all_df['smiles'].to_numpy()
smiles_np_nopains = smiles_np[~mask]

assays_nopains_df = pd.DataFrame(data = np.column_stack((smiles_np_nopains,assays_np_nopains)), columns = assay_all_df.columns)

In [ ]:
assays_nopains_df

In [ ]:
def check_col(df,col):
    if (df[col] != 1).all():
        return True
    else:
        return False

In [ ]:
assays_without_hits = []
for a in assays_nopains_df.columns[1:]:
    if check_col(assays_nopains_df, a):
        assays_without_hits.append(a)

In [ ]:
#remove assays without hits
assays_nopains_df.drop(columns=assays_without_hits, inplace=True)

In [ ]:
assays_nopains_df

In [ ]:
# Consider only hits
assays_np_nopains = assays_nopains_df.to_numpy()[:,1:]
assay_all_np_true = np.where(assays_np_nopains == 1,1,0)
plt.gcf().set_size_inches(15, 15)
sb.heatmap(assay_all_np_true)

In [ ]:
assays_np_nopains

In [ ]:
pca = PCA(.95)
pca_res = pca.fit_transform(assay_all_np_true)
a = pca_res[:,0:5]
principalDf = pd.DataFrame(data = a, columns = ['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5'])
plt.figure(figsize=(12,12))
color = sb.color_palette("hls", 5)
sb.scatterplot(data = principalDf, x="PCA 1", y="PCA 2", palette=color)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# intersection matrix, diagonal values are zeros
intersections = np.zeros((437,437))
assay_activations = []
for i in range(assay_all_np_true.shape[1]):
    assay_activations.append(set(np.argwhere(assay_all_np_true[:,i]==1).flatten()))
    
for i in range(437):
    for j in range(437):
        if i != j:
            intersections[i,j] = len(assay_activations[i].intersection(assay_activations[j]))
            intersections[j,i] = len(assay_activations[i].intersection(assay_activations[j]))
        
        
plt.gcf().set_size_inches(13, 13)    
sb.heatmap(intersections)

In [ ]:
sb.clustermap(intersections, metric = 'cosine')

In [ ]:
# intersection over union
intersections_relative = np.zeros((437,437))
assay_activations = []
for i in range(assay_all_np_true.shape[1]):
    assay_activations.append(set(np.argwhere(assay_all_np_true[:,i]==1).flatten()))
    
for i in range(437):
    for j in range(437):
        if len(assay_activations[i]) != 0 and len(assay_activations[j]) != 0:
            intersections_relative[i,j] = len(assay_activations[i].intersection(assay_activations[j])) / len(assay_activations[i].union(assay_activations[j]))
            intersections_relative[j,i] = len(assay_activations[i].intersection(assay_activations[j])) / len(assay_activations[i].union(assay_activations[j]))
            if intersections_relative[i,j] > 0.7:
                intersections_relative[i,j] = 1
            else:
                intersections_relative[i,j] = 0
            
            if intersections_relative[j,i] > 0.7:
                intersections_relative[j,i] = 1
            else:
                intersections_relative[j,i] = 0
        else:
            intersections_relative[i,j] = 0
            intersections_relative[j,i] = 0
        
        
plt.gcf().set_size_inches(13, 13)    
sb.heatmap(intersections_relative)

In [ ]:
#Clustering
plt.gcf().set_size_inches(17, 30)  
pl = sb.clustermap(intersections_relative, metric = 'cosine')

In [ ]:
intersections_relative.shape

In [ ]:

d = sch.distance.pdist(intersections_relative, metric = 'cosine')
L = sch.linkage(d, method='complete')
clusters = sch.fcluster(L, 0.95*d.max(), 'distance')

# clusters indicices correspond to incides of original df
print(max(clusters))
clusters_dict = {}
for i,cluster in enumerate(clusters):
    if cluster not in clusters_dict.keys():
        clusters_dict[cluster] = []
    
    clusters_dict[cluster].append(i)

#print(clusters_dict)
keep = []
for key in clusters_dict:
    if len(clusters_dict[key]) == 1:
        keep.append(clusters_dict[key][0])
    elif len(clusters_dict[key]) > 1:
        temp = random.sample(clusters_dict[key], int(len(clusters_dict[key])*0.3))
        for t in temp:
            keep.append(t)
            
print(len(keep))
f = intersections_relative[keep,:]
f = f[:,keep]
print(f.shape)
plt.gcf().set_size_inches(17, 30)  
pl2 = sb.clustermap(f, metric = 'cosine')

In [ ]:
assays_to_keep = assays_nopains_df.columns[keep]
assays_to_keep

In [ ]:
assays_nopains_df_filtered_assays = assays_nopains_df[assays_to_keep]
assays_nopains_df_filtered_assays

In [ ]:
assays_nopains_df_filtered_assays_np = assays_nopains_df_filtered_assays.to_numpy()[:,1:]
assays_nopains_df_filtered_assays_true = np.where(assays_nopains_df_filtered_assays_np == 1,1,0)
sums = np.sum(assays_nopains_df_filtered_assays_true, axis=1)
freq_compounds = [] 
for i in range(len(sums)):
    if sums[i] > 30:
        freq_compounds.append(i)

In [ ]:
assays_nopains_df_filtered_assays.drop(freq_compounds, inplace=True)

In [ ]:
assays_nopains_df_filtered_assays.reset_index(drop=True, inplace=True)

In [ ]:
assays_nopains_df_filtered_assays

In [ ]:
assays_without_hits2 = []
for a in assays_nopains_df_filtered_assays.columns[1:]:
    if check_col(assays_nopains_df_filtered_assays, a):
        assays_without_hits2.append(a)

In [ ]:
assays_nopains_df_filtered_assays_final = assays_nopains_df_filtered_assays.drop(columns=assays_without_hits2)
assays_nopains_df_filtered_assays_final

In [ ]:
assays_nopains_df_filtered_assays_final.to_csv('../data/assay_matrix_discrete_270_assays.csv', index=False)

In [ ]:
assays_nopains_df_filtered_assays_final = pd.read_csv('../data/assay_matrix_discrete_270_assays.csv')

In [ ]:
assays_nopains_df_filtered_assays_final

In [ ]:
assays_np_nopains_final = assays_nopains_df_filtered_assays_final.to_numpy()[:,1:]
assay_all_np_true_final = np.where(assays_np_nopains_final == 1,1,0)

# intersection matrix, diagonal values are zeros
# 270 is the number of assays obtained in the previous steps
intersections_final = np.zeros((270,270))
assay_activations_final = []
for i in range(assay_all_np_true_final.shape[1]):
    assay_activations_final.append(set(np.argwhere(assay_all_np_true_final[:,i]==1).flatten()))
    
for i in range(270):
    for j in range(270):
        if i != j:
            intersections_final[i,j] = len(assay_activations_final[i].intersection(assay_activations_final[j]))
            intersections_final[j,i] = len(assay_activations_final[i].intersection(assay_activations_final[j]))
        
        
plt.gcf().set_size_inches(13, 13)    
sb.heatmap(intersections_final)

In [ ]:
assays_np_nopains_final

In [ ]:
# intersection over union
intersections_relative_final = np.zeros((270,270))
assay_activations_final = []
for i in range(assay_all_np_true_final.shape[1]):
    assay_activations_final.append(set(np.argwhere(assay_all_np_true_final[:,i]==1).flatten()))
    
for i in range(270):
    for j in range(270):
        if len(assay_activations_final[i]) != 0 and len(assay_activations_final[j]) != 0:
            intersections_relative_final[i,j] = len(assay_activations_final[i].intersection(assay_activations_final[j])) / len(assay_activations_final[i].union(assay_activations_final[j]))
            intersections_relative_final[j,i] = len(assay_activations_final[i].intersection(assay_activations_final[j])) / len(assay_activations_final[i].union(assay_activations_final[j]))
        else:
            intersections_relative_final[i,j] = 0
            intersections_relative_final[j,i] = 0
        
        
plt.gcf().set_size_inches(13, 13)    
pl2 = sb.clustermap(intersections_relative_final, metric = 'cosine')

In [ ]:
ccc = []
cccc = set(assays_nopains_df_filtered_assays.columns) - set(assays_without_hits2)
for i in cccc:
    ccc.append(list(assays_nopains_df_filtered_assays.columns).index(i))

ff = f[:,ccc]
ff = ff[ccc,:]
plt.gcf().set_size_inches(17, 30)  
pl2 = sb.clustermap(ff, metric = 'cosine')

In [ ]:
# Make an IoU matrix with the order of element from clustering
intersections_relative_reorder = np.zeros((270,270))
for i in range(len(pl2.dendrogram_row.reordered_ind)):
    for j in range(len(pl2.dendrogram_row.reordered_ind)):
        intersections_relative_reorder[j,i] = intersections_relative_final[pl2.dendrogram_row.reordered_ind[j],pl2.dendrogram_row.reordered_ind[i]]
        intersections_relative_reorder[i,j] = intersections_relative_final[pl2.dendrogram_row.reordered_ind[i],pl2.dendrogram_row.reordered_ind[j]]

In [ ]:
plt.gcf().set_size_inches(13, 13) 
sb.heatmap(intersections_relative_reorder)

In [ ]:
assays_nopains_df_filtered_assays_final

In [ ]:
compounds16978 = assay_all_df['smiles'].to_list()

In [ ]:
#16 170 is the number of compounds left in the dataset
compounds16170 = assays_nopains_df_filtered_assays_final['smiles'].to_list()
indicies = []
for i in range(len(compounds16170)):
    indicies.append(compounds16978.index(compounds16170[i]))
    
print(len(indicies))


In [ ]:
# Commented out as the file is already there
# np.save('../data/compounds16978to16170.npy', indicies, allow_pickle=True)

In [ ]:
plt.gcf().set_size_inches(13, 13)
sb.heatmap(intersections_relative_final)